<a href="https://colab.research.google.com/github/parantou/OLLA/blob/Colab/10day_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U finance-datareader #colab에서 사용 시 실행 필요,

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from bs4 import BeautifulSoup
from urllib import parse
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import os
import requests
import re
import pandas as pd
import FinanceDataReader as fdr
# 버전 확인
fdr.__version__

'0.9.50'

In [3]:
#검색할 종목명 입력
stockName=input('종목명을 입력해주세요 : ')
print(stockName)

#csv파일로 저장해두거나 좀 더 빠르게 검색할 수 있도록 수정 필요
#코스피 종목 모두 가져오기
df_KOSPI = fdr.StockListing('KOSPI') 

#입력한 종목명의 종목코드 찾기
stockName=df_KOSPI.loc[df_KOSPI['Name'] == stockName]['Code'].values[0]
print(stockName)

종목명을 입력해주세요 : 기아
기아
000270


In [4]:
# 10일치 주가 데이터 추출
def getStockData(stockName):
    start_date= datetime.today() - timedelta(20)
    end_date= datetime.today()
    df = fdr.DataReader(stockName, start_date.strftime("%Y-%m-%d"),end_date.strftime("%Y-%m-%d"))
    df_ten = df.tail(10)
    df_ten.reset_index(inplace=True)
    #FinanceDataReader를 이용해 보조 데이터 가져오기
    df_dict = {
        'KS11' : fdr.DataReader('KS11', start_date, end_date), #코스피지수
        'KQ11' : fdr.DataReader('KQ11',start_date, end_date), #코스닥지수
        'DAU' : fdr.DataReader('DJI', start_date, end_date), #다우지수
        'NASDAQ' : fdr.DataReader('IXIC', start_date, end_date), #나스닥지수
        'SP500':  fdr.DataReader('US500', start_date, end_date), #S&P 500
        'USD' : fdr.DataReader('USD/KRW', start_date, end_date), #달러당 원화 환율
        'JPY' : fdr.DataReader('JPY/KRW', start_date, end_date) #엔화 원화 환율
    }

    #각 보조데이터의 수정주가를 추출해서 stock_data에 추가하기
    for key in df_dict:
      #df_dict[key] 인덱스 해제
      extra_df = df_dict[key].rename_axis('Date').reset_index()
      
      #사용할 칼럼명 생성
      newName= key+'_Adj_Close'

      #Date와 Adj Close컬럼만 추출
      extra_df_cut=extra_df.loc[:,['Date','Adj Close']]
      # print('======')
      # print(extra_df_cut)
      # print('======')
      #Adj Close컬럼명을 newName으로 변경
      extra_df_cut.rename(columns={'Adj Close':newName},inplace=True)

      #stock_data의 Date를 기준으로 extra_df_cut과 merge
      merge_outer = pd.merge(df_ten,extra_df_cut, how='left',left_on='Date', right_on='Date')
      
      #stock_data에 merge_outer의 newName(key+'_Adj_Close')칼럼 데이터 추가
      df_ten[newName] = merge_outer[newName]
      # df_ten.to_csv('test.csv')
    return df_ten

# 10일치 주가 데이터
stock_dataset=getStockData(stockName)
print(stock_dataset)

#결측치 확인
print(stock_dataset.isnull().sum())

#이전 행의 값으로 결측치 채우기
stock_dataset.fillna(method= 'ffill',inplace=True)

#결측치 확인
print(stock_dataset.isnull().sum()) #결측치 없음

        Date   Open   High    Low  Close   Volume    Change  KS11_Adj_Close  \
0 2022-12-12  65100  65600  65000  65100   687218 -0.001534     2373.020020   
1 2022-12-13  65300  65500  62100  62500  3121687 -0.039939     2372.399902   
2 2022-12-14  62200  64100  61800  64100  1960769  0.025600     2399.250000   
3 2022-12-15  63800  64800  63500  64100   878566  0.000000     2360.969971   
4 2022-12-16  63200  64400  63100  64100  1126438  0.000000     2360.020020   
5 2022-12-19  63700  63800  62000  62400  1296891 -0.026521     2352.169922   
6 2022-12-20  62000  62500  60900  62400  1458403  0.000000     2333.290039   
7 2022-12-21  62600  62700  61600  62000   810736 -0.006410     2328.949951   
8 2022-12-22  62000  63500  62000  63500  1004592  0.024194     2356.729980   
9 2022-12-23  62600  63300  62500  62700   765704 -0.012598     2313.689941   

   KQ11_Adj_Close  DAU_Adj_Close  NASDAQ_Adj_Close  SP500_Adj_Close  \
0      715.219971   34005.039063      11143.740234      399

<ipython-input-4-44a26e5d2f8f>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ten[newName] = merge_outer[newName]


In [7]:
!pip install transformers
from argparse import Namespace
from sklearn.metrics import classification_report
from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

import torch
import torch.nn as nn
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
url = ""
#(수정필요)삼성전자, 기아 등등... 종목코드 별로 if문 만들어서 url 돌아가도록 만들어야 함
if stockName == '000270' :
   url = "https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%E2%BE%C6&x=0&y=0&sm=title.basic&pd=4" #기아
# elif firstNumber == "016":
#     telecom = "KT"
# elif firstNumber == "019":
#     telecom = "LGU"
#else:

def financialNews(url):
    #컬럼스 : 종목명, 뉴스 제목 , 언론사, 작성날짜, 시간, 본문 내용
    stocklist=[]
    titlelist=[]
    companylist=[]
    datelist=[]
    totalsa_list=[] # 뉴스(제목) sa(sentiment analysis)
    # timelist=[]
    # contentlist=[]
    negative_list=[]
    positive_list=[]

    #header에 User Agent 정보(네이버의 경우 네이버 검색로봇 Yeti)를 기입하여 웹 서버로 페이지를 요청할 시에 같이 보내서 크롤링 제한을 피한다
    headers = {'User-Agent': 'Mozilla/5.0 (compatible; Yeti/1.1; +http://naver.me/spd)'}

    # 감성수치 사전학습 모델 로드
    tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
    model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")
    sa = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer) # 감성분류 모델 객체 sa 생성

    #i는 페이지 번호 | 1~200 반복
    for i in range(1,201) :
        #본문으로 들어가는 링크를 담을 리스트
        # links=[] 
        res = requests.get(url+str(i), headers = headers)
        soup = BeautifulSoup(res.text, 'html.parser') 
        body = soup.find(class_='newsSchResult')
        title = body.find_all(['dt','dd'], {'class': 'articleSubject'})  #뉴스제목
        
        #다음 페이지로 넘어갔을 때, 읽어올 title이 없을 경우 for문 종료
        if title == [] : break

        summary = body.find_all('dd', {'class': 'articleSummary'}) #요약문,언론사,날짜,시간
        stock = soup.select('p.resultCount > strong')[0].text  #종목명

        #한 페이지의 20개의 제목 가져오기
        for item in title:
          information = item.text 
          
          #link 추출 및 리스트에 저장
          # href = item.find('a').attrs['href']
          # links.append('https://finance.naver.com'+href)

          #제목 리스트에 저장
          titlelist.append(information.strip())
          #제목 가져올때마다 종목명 리스트안에 추가
          stocklist.append(stock) 
          
          # print('제목 : ',information)

        #한 페이지의 20개의 언론사 , 작성날짜, 시간 가져오기
        for item in summary:
          # information = item.string # 요약본, 언론사, 날짜, 시간
          press =  re.sub(r"\s", "", item.find('span', {'class': 'press'}).text) #언론사 -> 공백 제거완료
          wdate_all =  re.sub(r"\s", "", item.find('span', {'class': 'wdate'}).text) #날짜,시간 -> 공백 제거완료
          wdate=wdate_all[:10] #날짜만
          # time=wdate_all[10:] #시간만

          #언론사 리스트에 저장
          companylist.append(press)
          #작성날짜 리스트에 저장
          datelist.append(wdate) 
          #작성시간 리스트에 저장
          # timelist.append(time)

        #본문 추출
        '''
        for link in links:
          print('link : ',link)
          res = requests.get(link, headers = headers)
          soup = BeautifulSoup(res.text, 'html.parser')
          try:
              # link_news 내용이 있을 경우
              if soup.find('div',{'class': 'link_news'}) != None:
                  # link_news 제거
                  soup.find('div',{'class': 'link_news'}).decompose()
              #본문 내용 추출
              content = soup.find('div',{'id':'content'}).text.strip()
          except AttributeError as e:
              #본문이 없을 경우, NaN로 채우기
              print('error : ',e)

          print(content)
          print("-------다음뉴스------")
          #본문내용을 리스트에 저장
          contentlist.append(content)
        '''  
        #감성 수치 추출***************************************************
        # 사전학습 모델 로드
        # tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
        # model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")
        # sa = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer) # 감성분류 모델 객체 sa 생성

        #totalsa_list = [] # 뉴스(제목+내용) sa(sentiment analysis) 배열 초기화
    for news in titlelist:
        inputs = tokenizer(news, return_tensors='pt')
        output = model(**inputs)
        output = output.logits.tolist()[0]
        totalsa_list.append(output)
           
    outputs = torch.tensor(totalsa_list) # 출력값을 텐서로 변환 # outputs
    predictions = nn.functional.softmax(outputs, dim=-1) # 출력값을 확률 형태로 변환 # predictions
    sa = predictions.detach().numpy()
    # detach()는 tensor에서 이루어진 모든 연산을 분리한 tensor을 반환하는 method
    #print(predictions.detach().numpy())
    negative_list = sa[:, 0]
    positive_list = sa[:, 2]
    #print(negative_list)
    
    # 데이터셋 만들기
    NaverFinance = pd.DataFrame({
        'stock' : stocklist,
        'title' : titlelist,
        'company' : companylist,
        'Date' : datelist,
        # 'time' : timelist,
        # 'content' : contentlist
        'negative' : negative_list,
        'positive' : positive_list
    })

    NaverFinanceSI = NaverFinance.groupby(['Date']).mean()
    NaverFinanceSI['logit'] = np.log(NaverFinanceSI['positive']/NaverFinanceSI['negative']) # logit = ln(P/N)
    # print(NaverFinanceSI)
    # print(NaverFinanceSI['logit'].count())
    # print(NaverFinance['title'].count())
    avgn = NaverFinance['title'].count()/NaverFinanceSI['logit'].count() # 전체 기간에 작성된 뉴스 수의 평균
    NaverFinanceSI['intensity'] = np.log(1 + (NaverFinance.groupby(['Date'])['title'].count()/avgn))
    NaverFinanceSI['sent_index'] = NaverFinanceSI['logit'] * NaverFinanceSI['intensity']

    # #결과 출력
    # print(len(stocklist),len(titlelist),len(companylist),len(datelist),len(negative_list))       #,len(contentlist)
    print(NaverFinanceSI)
    # # csv 파일로 저장      
    # #NaverFinance.to_csv('NewFinance10.csv', mode='w', encoding='utf-8-sig') 
    return NaverFinanceSI

if __name__ == "__main__":
      start_date=stock_dataset.head(1)['Date'][0].strftime("%Y-%m-%d")
      end_date=stock_dataset.tail(1)['Date'][9].strftime("%Y-%m-%d")

      financialNews(url +"&stDateStart="+start_date+"&stDateEnd="+end_date+"&page=")
      NaverFinanceSI_dataset = financialNews(url +"&stDateStart="+start_date+"&stDateEnd="+end_date+"&page=")
      

            negative  positive     logit  intensity  sent_index
Date                                                           
2022-12-12  0.215731  0.611738  1.042272   0.648695    0.676117
2022-12-13  0.503908  0.333385 -0.413095   0.330242   -0.136421
2022-12-14  0.798041  0.182664 -1.474513   0.231802   -0.341794
2022-12-15  0.324702  0.557522  0.540594   0.461818    0.249656
2022-12-16  0.071929  0.551480  2.036925   1.038893    2.116147
2022-12-19  0.406070  0.100716 -1.394220   0.991640   -1.382564
2022-12-20  0.332561  0.020789 -2.772393   0.889857   -2.467035
2022-12-21  0.042555  0.675532  2.764700   0.942043    2.604467
2022-12-22  0.079995  0.245790  1.122518   0.282232    0.316811
            negative  positive     logit  intensity  sent_index
Date                                                           
2022-12-12  0.215731  0.611738  1.042272   0.648695    0.676117
2022-12-13  0.503908  0.333385 -0.413095   0.330242   -0.136421
2022-12-14  0.798041  0.182664 -1.474513

In [9]:
#NaverFinanceSI_dataset
#NaverFinanceSI_dataset['Date','stock'] = NaverFinanceSI_dataset.index
NaverFinanceSI_dataset.reset_index(inplace=True)
NaverFinanceSI_dataset['Date'] = pd.to_datetime(NaverFinanceSI_dataset['Date'])

In [11]:
#주가랑 보조 합침
#뉴스 제목만 추출

#감성지수 추출 필요
#감성 지수랑 주가+보조 데이터 합쳐야함
#합친걸로 모델에 넣기

NaverFinanceF = pd.merge(stock_dataset,NaverFinanceSI_dataset, on='Date', how='outer').drop(columns=['negative','positive','logit'])

NaverFinanceF = NaverFinanceF.fillna(0)
NaverFinanceF

,Date,Open,High,Low,Close,Volume,Change,KS11_Adj_Close,KQ11_Adj_Close,DAU_Adj_Close,NASDAQ_Adj_Close,SP500_Adj_Close,USD_Adj_Close,JPY_Adj_Close,intensity,sent_index
0,2022-12-12,65100,65600,65000,65100,687218,-0.001534,2373.020020,715.219971,34005.039063,11143.740234,3990.560059,1303.170044,9.523555,0.648695,0.676117
1,2022-12-13,65300,65500,62100,62500,3121687,-0.039939,2372.399902,715.159973,34108.640625,11256.809570,4019.649902,1305.920044,9.497186,0.330242,-0.136421
2,2022-12-14,62200,64100,61800,64100,1960769,0.025600,2399.250000,729.000000,33966.351563,11170.889648,3995.320068,1288.979980,9.507715,0.231802,-0.341794
3,2022-12-15,63800,64800,63500,64100,878566,0.000000,2360.969971,722.679993,33202.218750,10810.530273,3895.750000,1295.680054,9.571786,0.461818,0.249656
4,2022-12-16,63200,64400,63100,64100,1126438,0.000000,2360.020020,717.409973,32920.460938,10705.410156,3852.360107,1317.099976,9.561793,1.038893,2.116147
5,2022-12-19,63700,63800,62000,62400,1296891,-0.026521,2352.169922,717.219971,32757.539063,10546.030273,3817.659912,1308.859985,9.584360,0.991640,-1.382564
6,2022-12-20,62000,62500,60900,62400,1458403,0.000000,2333.290039,703.130005,32849.738281,10547.110352,3821.620117,1301.329956,9.499869,0.889857,-2.467035
7,2022-12-21,62600,62700,61600,62000,810736,-0.006410,2328.949951,705.700012,33376.480469,10709.370117,3878.439941,1283.640015,9.732287,0.942043,2.604467
8,2022-12-22,62000,63500,62000,63500,1004592,0.024194,2356.729980,715.020020,33027.488281,10476.120117,3822.389893,1280.770020,9.675316,0.282232,0.316811
9,2022-12-23,62600,63300,62500,62700,765704,-0.012598,2313.689941,691.250000,33203.929688,10497.860352,3844.820068,1290.199951,9.747290,0.000000,0.000000
